In [532]:
from typing import TypedDict, List
import random
from langgraph.graph import StateGraph, START, END

In [533]:
class AgentState(TypedDict):
  name: str
  age: int
  number: List[int]
  counter: int
  message: str

In [534]:
def greeter(state: AgentState) -> AgentState:
  """
  A simple func that greets the user
  """
  state["message"] = f"Hello {state["name"]}, How are you doing today?"
  
  return state


def age_doubler(state: AgentState) -> AgentState:
  """
  This func doubles the age of the user
  """
  state["message"] = state["message"] + f"Your new age is now: {state["age"] * 2}!"
  state["counter"] = 0
  return state

def random_number_generator(state: AgentState) -> AgentState:
  """
  This func generates random_numbers using random func
  """
  # print("random_number_generator func")
  state["number"].append(random.randint(0, 10))
  state["counter"] += 1

  return state


def should_continue(state: AgentState) -> AgentState:
  """
  This func routes to the node that satisfies the given condition
  """
  # print("currently in the should continue func, the count value is:", COUNTER)
  if state["counter"] < 5:
    print("Entering Loop", state["counter"])
    return "loop"
  else:
    return "exit"


In [535]:
graph = StateGraph(AgentState)

graph.add_node("greeting_node", greeter)
graph.add_node("age_doubling_node", age_doubler)
graph.add_node("randomizer_node", random_number_generator)
graph.add_node("end_node", lambda state: state)

graph.set_entry_point("greeting_node")
graph.add_edge("greeting_node", "age_doubling_node")
graph.add_edge("age_doubling_node", "randomizer_node")
graph.add_conditional_edges(
  "randomizer_node",
  should_continue,
  {
      "loop": "randomizer_node",
      "exit": "end_node"
  }
)

graph.set_finish_point("end_node")

app = graph.compile()

In [536]:
  #  name: str
  #   age: int
  #   number: List[int]
  #   counter: int
  #   flag: bool
  #   message: str

rslt = app.invoke(
    {
        "name": "wrichy",
        "age": 20, 
        "number": [],
        "counter": 7
    })

print(rslt)

Entering Loop 1
Entering Loop 2
Entering Loop 3
Entering Loop 4
{'name': 'wrichy', 'age': 20, 'number': [3, 10, 6, 1, 8], 'counter': 5, 'message': 'Hello wrichy, How are you doing today?Your new age is now: 40!'}


## An Automatic Higer or Lower Game

In [537]:
class GameState(TypedDict):
    name: str
    guesses: List[int]
    # unique_guesses: List[int]
    lucky_no: int
    counter: int
    attempts: int
    flag: bool
    lower_bound: int
    upper_bound: int
    rslt_message: str


In [538]:
### This game graph is a guessing game where there is no human in the loop ###
###How it works: a lucky no is generated by the calibration node and this no is meant to be guessed by the guess node
### If the guessed value is higher or lower than the lucky no, then it readjust till it guesses the lucky no
### Constraint: The framework sees guess greater than 25 attempts as a recursive Error which isnt

def setup_game(state: GameState) -> GameState:
    state["rslt_message"] = f"Hello {state["name"]}, welcome to guessIt"
    state["counter"] = 0
    state["flag"] = False

    return state

def lucky_no_calibrator(state: GameState) -> GameState:
  """
  produces a new random guess value if the previous guess value is out of range between the lower and upper bound values
  """
  state["lucky_no"] = random.randint(1, 100)

  return state

def guess_number(state: GameState) -> GameState:
    """
    function implements the guess logic
    """
    guess = random.randint(state["lower_bound"], state["upper_bound"])

    if 1 < state["attempts"] > 25:
        raise ValueError("Invalid Input, choose Attempts value greater than 1 or less than 25")
    
    if guess not in state["guesses"]:
      state["guesses"].append(guess)
      state["counter"] += 1
    if guess == state["lucky_no"]:
      print(f"I have guessed it right, after {state["counter"]} attempts, the lucky no is: {state["lucky_no"]}\
 and my guess was {guess}, hence i was right!")
      state["flag"] = True
      return state
        
    elif guess > state["lucky_no"]:
        state["upper_bound"] = guess
        print("guess: ", guess)

    else:
        state["lower_bound"] = guess
        print("guess: ", guess)
      
    
    return state

def should_continue_calibrating(state: GameState) -> GameState:
    if state["upper_bound"] - state["lower_bound"] < 10:
        raise ValueError("The range between the upper_bound value and the lower_bound value must be greater than or equal to 10")
    
    if (state["lower_bound"] >= state["lucky_no"]) or (state["lucky_no"] >= state["upper_bound"]):
      print(f"Lucky no is, {state["lucky_no"]} it seems lucky no is not in the range of the upper_bound and lower_bound value, recalibrating...")
      return "loop"

    else:
      return "exit"
    
def should_continue_guessing(state: GameState) -> GameState:
    # if state["lower_bound"] < 0  and state["upper_bound"] > 100:
    #     raise ValueError("Sorry wrong input, either selected lower_bound or upper_bound value out of range\
    #                      Choose a lower_bound value greater than 0 and upper_bound value less than 100")
    
    if (state["counter"] <= state["attempts"]) and (state["flag"] == False):
        print("Guessing again, it seems i guessed wrong", state["guesses"])
        return "loop"
    else:
        return "exit"

In [539]:
game_graph = StateGraph(GameState)

game_graph.add_node("setup_node", setup_game)
game_graph.add_node("calibration_node", lucky_no_calibrator)
game_graph.add_node("guess_node", guess_number)
game_graph.add_node("end_of_calibration_node", lambda state: state)
game_graph.add_node("end_of_guess_node", lambda state: state)
game_graph.set_entry_point("setup_node")
game_graph.add_edge("setup_node", "calibration_node")
game_graph.add_conditional_edges(
      "calibration_node",
      should_continue_calibrating,
      {
        "loop":"calibration_node",
        "exit":"end_of_calibration_node"
      }
)
# game_graph.add_edge("calibration_node", "end_of_calibration_node") # Not sure about this line yet
game_graph.add_edge("end_of_calibration_node", "guess_node")
game_graph.add_conditional_edges(
      "guess_node",
      should_continue_guessing,
      {
        "loop":"guess_node",
        "exit":"end_of_guess_node"
      }
)

game_graph.set_finish_point("end_of_guess_node")

game_app = game_graph.compile()

In [544]:
rslt = game_app.invoke(
    {
        "name": "wrichy",
        "guesses": [],
        "attempts": 24,
        "lower_bound": -13,
        "upper_bound": 3
    }
)

Lucky no is, 90 it seems lucky no is not in the range of the upper_bound and lower_bound value, recalibrating...
Lucky no is, 22 it seems lucky no is not in the range of the upper_bound and lower_bound value, recalibrating...
Lucky no is, 7 it seems lucky no is not in the range of the upper_bound and lower_bound value, recalibrating...
Lucky no is, 92 it seems lucky no is not in the range of the upper_bound and lower_bound value, recalibrating...
Lucky no is, 10 it seems lucky no is not in the range of the upper_bound and lower_bound value, recalibrating...
Lucky no is, 31 it seems lucky no is not in the range of the upper_bound and lower_bound value, recalibrating...
Lucky no is, 44 it seems lucky no is not in the range of the upper_bound and lower_bound value, recalibrating...
Lucky no is, 46 it seems lucky no is not in the range of the upper_bound and lower_bound value, recalibrating...
Lucky no is, 33 it seems lucky no is not in the range of the upper_bound and lower_bound value, r

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT